In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.getOrCreate()

23/10/31 11:16:07 WARN Utils: Your hostname, Priyanshs-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.30.200 instead (on interface en0)
23/10/31 11:16:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/31 11:16:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/10/31 11:16:07 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [14]:
data = spark.read.csv('Iris.csv', inferSchema=True, header=True)
data.show(5)

+-------------+------------+-------------+------------+-------+
|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|Species|
+-------------+------------+-------------+------------+-------+
|          5.1|         3.5|          1.4|         0.2|      0|
|          4.9|         3.0|          1.4|         0.2|      0|
|          4.7|         3.2|          1.3|         0.2|      0|
|          4.6|         3.1|          1.5|         0.2|      0|
|          5.0|         3.6|          1.4|         0.2|      0|
+-------------+------------+-------------+------------+-------+
only showing top 5 rows



In [15]:
data.printSchema()

root
 |-- SepalLengthCm: double (nullable = true)
 |-- SepalWidthCm: double (nullable = true)
 |-- PetalLengthCm: double (nullable = true)
 |-- PetalWidthCm: double (nullable = true)
 |-- Species: integer (nullable = true)



In [16]:
data.columns

['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm', 'Species']

In [17]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm'], outputCol='features')
featured_data = assembler.transform(data)
featured_data.show(5)

+-------------+------------+-------------+------------+-------+-----------------+
|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|Species|         features|
+-------------+------------+-------------+------------+-------+-----------------+
|          5.1|         3.5|          1.4|         0.2|      0|[5.1,3.5,1.4,0.2]|
|          4.9|         3.0|          1.4|         0.2|      0|[4.9,3.0,1.4,0.2]|
|          4.7|         3.2|          1.3|         0.2|      0|[4.7,3.2,1.3,0.2]|
|          4.6|         3.1|          1.5|         0.2|      0|[4.6,3.1,1.5,0.2]|
|          5.0|         3.6|          1.4|         0.2|      0|[5.0,3.6,1.4,0.2]|
+-------------+------------+-------------+------------+-------+-----------------+
only showing top 5 rows



In [18]:
final_data = featured_data.select('features','Species')
final_data.show(5)

+-----------------+-------+
|         features|Species|
+-----------------+-------+
|[5.1,3.5,1.4,0.2]|      0|
|[4.9,3.0,1.4,0.2]|      0|
|[4.7,3.2,1.3,0.2]|      0|
|[4.6,3.1,1.5,0.2]|      0|
|[5.0,3.6,1.4,0.2]|      0|
+-----------------+-------+
only showing top 5 rows



In [19]:
train_data, test_data = final_data.randomSplit([0.8,0.2])

In [20]:
train_data.count()

112

In [21]:
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [22]:
classifier = DecisionTreeClassifier(labelCol='Species').fit(train_data)

In [23]:
predictions = classifier.transform(test_data)
predictions.show()

+-----------------+-------+--------------+--------------------+----------+
|         features|Species| rawPrediction|         probability|prediction|
+-----------------+-------+--------------+--------------------+----------+
|[4.4,3.0,1.3,0.2]|      0|[39.0,0.0,0.0]|       [1.0,0.0,0.0]|       0.0|
|[4.4,3.2,1.3,0.2]|      0|[39.0,0.0,0.0]|       [1.0,0.0,0.0]|       0.0|
|[4.6,3.2,1.4,0.2]|      0|[39.0,0.0,0.0]|       [1.0,0.0,0.0]|       0.0|
|[4.6,3.6,1.0,0.2]|      0|[39.0,0.0,0.0]|       [1.0,0.0,0.0]|       0.0|
|[5.0,2.0,3.5,1.0]|      1|[0.0,36.0,1.0]|[0.0,0.9729729729...|       1.0|
|[5.0,3.0,1.6,0.2]|      0|[39.0,0.0,0.0]|       [1.0,0.0,0.0]|       0.0|
|[5.0,3.5,1.6,0.6]|      0|[39.0,0.0,0.0]|       [1.0,0.0,0.0]|       0.0|
|[5.1,3.5,1.4,0.3]|      0|[39.0,0.0,0.0]|       [1.0,0.0,0.0]|       0.0|
|[5.1,3.8,1.5,0.3]|      0|[39.0,0.0,0.0]|       [1.0,0.0,0.0]|       0.0|
|[5.4,3.0,4.5,1.5]|      1|[0.0,36.0,1.0]|[0.0,0.9729729729...|       1.0|
|[5.4,3.7,1.5,0.2]|      

In [24]:
model_accuracy = MulticlassClassificationEvaluator(labelCol='Species',metricName='accuracy').evaluate(predictions)
model_accuracy*100

97.36842105263158

In [3]:
raw_data = spark.read.csv('Obesity Classification.csv', header=True, inferSchema=True)
raw_data.show(5)

+---+---+------+------+------+----+-------------+
| ID|Age|Gender|Height|Weight| BMI|        Label|
+---+---+------+------+------+----+-------------+
|  1| 25|  Male|   175|    80|25.3|Normal Weight|
|  2| 30|Female|   160|    60|22.5|Normal Weight|
|  3| 35|  Male|   180|    90|27.3|   Overweight|
|  4| 40|Female|   150|    50|20.0|  Underweight|
|  5| 45|  Male|   190|   100|31.2|        Obese|
+---+---+------+------+------+----+-------------+
only showing top 5 rows



In [4]:
raw_data.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Height: integer (nullable = true)
 |-- Weight: integer (nullable = true)
 |-- BMI: double (nullable = true)
 |-- Label: string (nullable = true)



In [7]:
raw_data.groupby('Label').count().show()

+-------------+-----+
|        Label|count|
+-------------+-----+
|Normal Weight|   29|
|   Overweight|   20|
|  Underweight|   47|
|        Obese|   12|
+-------------+-----+



In [8]:
raw_data.count()

108

In [28]:
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCols=['Gender','Label'],outputCols=['Gender_ind','Label_ind'])
indexed_data1 = indexer.fit(raw_data).transform(raw_data)
indexed_data1.show(5)

+---+---+------+------+------+----+-------------+----------+---------+
| ID|Age|Gender|Height|Weight| BMI|        Label|Gender_ind|Label_ind|
+---+---+------+------+------+----+-------------+----------+---------+
|  1| 25|  Male|   175|    80|25.3|Normal Weight|       0.0|      1.0|
|  2| 30|Female|   160|    60|22.5|Normal Weight|       1.0|      1.0|
|  3| 35|  Male|   180|    90|27.3|   Overweight|       0.0|      2.0|
|  4| 40|Female|   150|    50|20.0|  Underweight|       1.0|      0.0|
|  5| 45|  Male|   190|   100|31.2|        Obese|       0.0|      3.0|
+---+---+------+------+------+----+-------------+----------+---------+
only showing top 5 rows



In [29]:
indexed_data1.groupby('Label_ind').count().show()

+---------+-----+
|Label_ind|count|
+---------+-----+
|      0.0|   47|
|      1.0|   29|
|      3.0|   12|
|      2.0|   20|
+---------+-----+



In [30]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=['Age','Gender_ind','Height','Weight','BMI'], outputCol='features')
featured_data1 = assembler.transform(indexed_data1)
featured_data1.show(5)

+---+---+------+------+------+----+-------------+----------+---------+--------------------+
| ID|Age|Gender|Height|Weight| BMI|        Label|Gender_ind|Label_ind|            features|
+---+---+------+------+------+----+-------------+----------+---------+--------------------+
|  1| 25|  Male|   175|    80|25.3|Normal Weight|       0.0|      1.0|[25.0,0.0,175.0,8...|
|  2| 30|Female|   160|    60|22.5|Normal Weight|       1.0|      1.0|[30.0,1.0,160.0,6...|
|  3| 35|  Male|   180|    90|27.3|   Overweight|       0.0|      2.0|[35.0,0.0,180.0,9...|
|  4| 40|Female|   150|    50|20.0|  Underweight|       1.0|      0.0|[40.0,1.0,150.0,5...|
|  5| 45|  Male|   190|   100|31.2|        Obese|       0.0|      3.0|[45.0,0.0,190.0,1...|
+---+---+------+------+------+----+-------------+----------+---------+--------------------+
only showing top 5 rows



In [31]:
final_data1 = featured_data1.select('features', 'Label_ind')
final_data1.show(5)

+--------------------+---------+
|            features|Label_ind|
+--------------------+---------+
|[25.0,0.0,175.0,8...|      1.0|
|[30.0,1.0,160.0,6...|      1.0|
|[35.0,0.0,180.0,9...|      2.0|
|[40.0,1.0,150.0,5...|      0.0|
|[45.0,0.0,190.0,1...|      3.0|
+--------------------+---------+
only showing top 5 rows



In [35]:
final_data1.count()

108

In [36]:
train, test = final_data1.randomSplit([0.8,0.2])
train.count()

92

In [37]:
test.count()

16

In [38]:
classifier1 = DecisionTreeClassifier(labelCol='Label_ind').fit(train)

In [39]:
predictions1 = classifier1.transform(test)
predictions1.show(10)

+--------------------+---------+------------------+--------------------+----------+
|            features|Label_ind|     rawPrediction|         probability|prediction|
+--------------------+---------+------------------+--------------------+----------+
|[12.0,0.0,175.0,1...|      0.0|[41.0,0.0,0.0,0.0]|   [1.0,0.0,0.0,0.0]|       0.0|
|[16.0,1.0,160.0,1...|      0.0|[41.0,0.0,0.0,0.0]|   [1.0,0.0,0.0,0.0]|       0.0|
|[17.0,1.0,160.0,1...|      0.0|[41.0,0.0,0.0,0.0]|   [1.0,0.0,0.0,0.0]|       0.0|
|[33.0,0.0,190.0,4...|      0.0|[41.0,0.0,0.0,0.0]|   [1.0,0.0,0.0,0.0]|       0.0|
|[36.0,0.0,190.0,7...|      1.0|[0.0,21.0,0.0,0.0]|   [0.0,1.0,0.0,0.0]|       1.0|
|[39.0,0.0,190.0,9...|      2.0|[0.0,1.0,16.0,0.0]|[0.0,0.0588235294...|       2.0|
|[50.0,1.0,130.0,6...|      1.0|[0.0,21.0,0.0,0.0]|   [0.0,1.0,0.0,0.0]|       1.0|
|[53.0,0.0,210.0,5...|      1.0|[0.0,21.0,0.0,0.0]|   [0.0,1.0,0.0,0.0]|       1.0|
|[56.0,0.0,210.0,9...|      2.0|[0.0,1.0,16.0,0.0]|[0.0,0.0588235294...|    

In [41]:
acc = MulticlassClassificationEvaluator(labelCol='Label_ind',metricName='accuracy').evaluate(predictions1)
acc*100

100.0